# Pawfect Planner – Your Personalized Pet Service and Activity Recommender


### Import Required Libraries

In [ ]:
import requests
import requests_cache
import pandas as pd

### Initialize Caching
Initialize cache to store API responses for 1 hour

In [25]:
requests_cache.install_cache('pawfect_cache', expire_after=3600)

### Define API Keys

In [26]:
GOOGLE_PLACES_API_KEY = ''
OPENWEATHERMAP_API_KEY = ''

### Convert a city name or postal code into latitude and longitude using Google Geocoding API

In [27]:
def get_lat_lon_from_location(location, api_key):
    geo_url = "https://maps.googleapis.com/maps/api/geocode/json"
    params = {
        'address': location,
        'key': api_key
    }
    response = requests.get(geo_url, params=params)
    
    if response.status_code == 200:
        data = response.json()
        if data.get('results'):
            lat_lng = data['results'][0]['geometry']['location']
            return f"{lat_lng['lat']},{lat_lng['lng']}"
        else:
            print("No results found for the given location.")
            return None
    else:
        print("Error fetching geolocation data.")
        return None

### Fetch Pet-Friendly Places Using Google Places API

In [30]:
def get_pet_friendly_places(location, keywords=['dog park', 'pet store', 'veterinary clinic', 'pet-friendly café', 'pet grooming', 'pet-friendly hotel'], 
                            min_rating=3.5, radius=5000):
    
    ## We only want places that have ratings higher than 3.5 stars
    
    """
    Parameters:
    location (str): City name or ZIP code
    keywords (list): General pet-friendly places to search
    min_rating (float): Minimum acceptable rating (default: 3.5)
    radius (int): Search radius in meters (default: 5000)
    
    Returns:
    DataFrame: List of places with name, address, rating, and category
    """
    
    lat_lon = get_lat_lon_from_location(location, GOOGLE_PLACES_API_KEY)
    if not lat_lon:
        return None
    
    places = []
    
    for keyword in keywords:
        url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
        params = {
            'location': lat_lon,
            'radius': radius,
            'keyword': keyword,
            'key': GOOGLE_PLACES_API_KEY
        }
        
        response = requests.get(url, params=params)
        
        if response.status_code == 200:
            data = response.json()
            for place in data.get('results', []):
                rating = place.get('rating', 0)
                
                # Apply rating filter
                
                if rating >= min_rating:
                    places.append({
                        'Name': place['name'],
                        'Address': place.get('vicinity', 'N/A'),
                        'Rating': rating,
                        'Category': keyword  
                    })
        else:
            print(f"Error fetching data from Google Places API for {keyword}")
    
    return pd.DataFrame(places) if places else pd.DataFrame()

### Fetch Weather Information Using OpenWeatherMap API

In [32]:
def get_weather(location):
    url = "http://api.openweathermap.org/data/2.5/weather"
    if location.isdigit():
        params = {'zip': f"{location},US", 'appid': OPENWEATHERMAP_API_KEY, 'units': 'metric'}
    else:
        params = {'q': location, 'appid': OPENWEATHERMAP_API_KEY, 'units': 'metric'}

    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        weather_info = {
            'City': data.get('name', 'Unknown City'), 
            'Temperature': data['main']['temp'],
            'Weather': data['weather'][0]['description'],
            'Wind Speed': data['wind']['speed']
        }
        return weather_info
    else:
        print("Error fetching data from OpenWeatherMap API")
        return {}

### Display Pet-Friendly Places Grouped by Category

In [33]:
def display_pet_friendly_places(places_df):
    if not places_df.empty:
        print("\n📌 Recommended Pet-Friendly Places:\n")
        
        # Group places by category
        grouped_places = places_df.groupby("Category")

        for category, places in grouped_places:
            print(f"\n🐾 **{category.upper()}** ({len(places)} locations found):")
            
            # Show only up to 10 results per category
            
            for index, (_, row) in enumerate(places.iterrows()):
                if index < 10:  
                    print(f"  {index + 1}. {row['Name']} - {row['Address']} (⭐ {row['Rating']})")
            print("-" * 50)  
            
    else:
        print("❌ No pet-friendly places found. Try searching for a different location.")

### Display Weather Data in a Readable Format

In [35]:
def display_weather_info(weather_info, user_location):
    if weather_info:
        print(f"\n🌦️ **Current Weather in {user_location}:**")
        print(f"   🌡️ Temperature: {weather_info.get('Temperature', 'N/A')}°C")
        print(f"   ☁️ Weather: {weather_info.get('Weather', 'N/A').capitalize()}") 
        print(f"   💨 Wind Speed: {weather_info.get('Wind Speed', 'N/A')} km/h")
        print("-" * 50)  
    else:
        print("\n❌ Weather information could not be retrieved.")

### Integrate Pet-Friendly Places and Weather Data

In [36]:
def pawfect_planner(user_location):
    print(f"\n📍 Fetching pet-friendly places near **{user_location}**...")
    places_df = get_pet_friendly_places(user_location)

    print(f"\n🌦️ Fetching weather information for **{user_location}**...")
    weather_info = get_weather(user_location)

    # Display Weather Information
    display_weather_info(weather_info, user_location)

    # Display Pet-Friendly Places
    display_pet_friendly_places(places_df)


### Get User Input & Run the Final Program

In [37]:
if __name__ == "__main__":
    user_input = input("\n🔎 Enter city name or postal code: ")
    pawfect_planner(user_input)


🔎 Enter city name or postal code: 94103

📍 Fetching pet-friendly places near **94103**...

🌦️ Fetching weather information for **94103**...

🌦️ **Current Weather in 94103:**
   🌡️ Temperature: 13.5°C
   ☁️ Weather: Few clouds
   💨 Wind Speed: 5.66 km/h
--------------------------------------------------

📌 Recommended Pet-Friendly Places:


🐾 **DOG PARK** (20 locations found):
  1. Channel Street Dog Park - 738 Long Bridge Street, San Francisco (⭐ 4.6)
  2. Mountain Lake Park Dog Play Area - 1 Funston Ave, San Francisco (⭐ 4.8)
  3. Berry Street Dog Park - 451 Berry St, San Francisco (⭐ 4.3)
  4. Patrick's Park - Fort Mason, San Francisco (⭐ 4.8)
  5. Walter Haas Dog Play Area - 8515 Diamond Heights Blvd, San Francisco (⭐ 4.5)
  6. Rincon Hill Dog Park - Bryant St & Beale St, San Francisco (⭐ 3.9)
  7. Yerba Buena Island Dog Park - 60 Yerba Buena Rd, San Francisco (⭐ 4.8)
  8. Duboce Park - Duboce Park, Scott St, San Francisco (⭐ 4.6)
  9. Mission Creek Dog Park - 465-479 Berry St, San